In [1]:
!kaggle competitions download -c spaceship-titanic

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

/var/folders/83/ypn030093497vyfy0wz4lyqc0000gn/T/ipykernel_49371/1208835502.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# First I am going for a very simple model with dropping all the nan values and converting the categorical objects into dummy

In [230]:
from torch.utils.data import Dataset
class tit_space(Dataset):
    def __init__(self, to_ = 'train'):
        df = pd.read_csv(f'spaceship-titanic/{to_}.csv')

        df.dropna(inplace = True)
        df.drop(['Name', 'PassengerId', 'Cabin'], axis = 1, inplace = True)
        df = pd.get_dummies(df, columns = ['HomePlanet', 'Destination'], drop_first = True)
        df.replace([False, True], [0,1], inplace = True)
        df = (df-df.min())/(df.max() - df.min())
        # print(df)
        self.label = torch.tensor(df.pop('Transported').to_numpy(), dtype = torch.float)
        self.df = torch.tensor(df.to_numpy(), dtype = torch.float)
    
    def __getitem__(self, idx):
        return self.df[idx], self.label[idx]
    
    def __len__(self):
        return self.df.shape[0]


In [231]:
torch.manual_seed(42)
from torch.utils.data import DataLoader
from torch.utils.data import random_split

tit_data = tit_space()
train_dataset, test_dataset = random_split(tit_data, [0.9, 0.1])
len(train_dataset), len(test_dataset)
tit_data[0]

      CryoSleep       Age  VIP  RoomService  FoodCourt  ShoppingMall  \
0           0.0  0.493671  0.0     0.000000   0.000000      0.000000   
1           0.0  0.303797  0.0     0.010988   0.000302      0.002040   
2           0.0  0.734177  1.0     0.004335   0.119948      0.000000   
3           0.0  0.417722  0.0     0.000000   0.043035      0.030278   
4           0.0  0.202532  0.0     0.030544   0.002348      0.012324   
...         ...       ...  ...          ...        ...           ...   
8688        0.0  0.518987  1.0     0.000000   0.228726      0.000000   
8689        1.0  0.227848  0.0     0.000000   0.000000      0.000000   
8690        0.0  0.329114  0.0     0.000000   0.000000      0.152779   
8691        0.0  0.405063  0.0     0.000000   0.035186      0.000000   
8692        0.0  0.556962  0.0     0.012702   0.157247      0.000000   

           Spa    VRDeck  Transported  HomePlanet_Europa  HomePlanet_Mars  \
0     0.000000  0.000000          0.0                1.0  

/var/folders/83/ypn030093497vyfy0wz4lyqc0000gn/T/ipykernel_49371/3488158696.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace([False, True], [0,1], inplace = True)


(tensor([0.0000, 0.4937, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 1.0000]),
 tensor(0.))

In [314]:
# Hyperparameter----

batch_size = 32
lr = 1e-4
epoch = 11
batch_eval_inter = 100
eval_train_test = 10
# eval_test = 10

# ------------------

In [267]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [304]:
# Model
torch.manual_seed(42)
class space_finder(nn.Module):
    def __init__(self):
        super(space_finder, self).__init__()
        # self.block = nn.Sequential(
        #     nn.Linear(12, 100),
        #     nn.BatchNorm1d(100),
        #     nn.ReLU(),
        #     nn.Linear(100, 200),
        #     nn.BatchNorm1d(200),
        #     nn.ReLU(),
        #     nn.Linear(200, 500),
        #     nn.BatchNorm1d(100),
        #     nn.ReLU(),
        #     nn.Linear(100, 50),
        #     nn.BatchNorm1d(50),
        #     nn.ReLU(),
        #     nn.Linear(50, 1),
        # )
        self.l1 = nn.Sequential(
                nn.Linear(12, ),
                nn.BatchNorm1d(100),
                nn.ReLU())
        
        self.l2 = nn.Sequential(
                nn.Linear(100, 200),
                nn.BatchNorm1d(200),
                nn.ReLU())
        self.l3 = nn.Sequential(
                nn.Linear(200, 500),
                nn.BatchNorm1d(500),
                nn.ReLU())
        self.l4 = nn.Sequential(
                nn.Linear(500, 200),
                nn.BatchNorm1d(200),
                nn.ReLU())
        self.l5 = nn.Sequential(
                nn.Linear(200, 100),
                nn.BatchNorm1d(100),
                nn.ReLU())
        self.logits = nn.Linear(100, 1)

    def forward(self, x):
        
        x1 = self.l1(x)
        x2 = self.l2(x1)
        x = self.l3(x2)
        x = x2 + self.l4(x)
        x = x1 + self.l5(x)
        logits = self.logits(x)
        return logits
        

In [305]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [306]:
model = space_finder()
model.to(device)

space_finder(
  (l1): Sequential(
    (0): Linear(in_features=12, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=100, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=200, out_features=500, bias=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (l4): Sequential(
    (0): Linear(in_features=500, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (l5): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (logits): Linear(in_featu

In [307]:
#experimentation
# torch.manual_seed(42)
# x = torch.rand((1,5,12), device = device)
# y = model(x)
# y

In [308]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [318]:
import time
from sklearn.metrics import accuracy_score

def train_loop(i, see_batch_loss = False):
    model.train()
    total_loss = 0
    y_true = []
    y_preds = []
    for batch, (data, label) in enumerate(train_loader):
        data , label = data.to(device), label.to(device)
        # print(data.shape)
        logits = model(data)
        # preds = preds.argmax(dim = -1)
        # print(logits)
        optimizer.zero_grad()
        loss = loss_fn(logits.view(-1), label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.sigmoid(logits).view(-1) > 0.5
        y_true.extend(label.cpu().tolist())
        y_preds.extend(preds.detach().cpu().tolist())
        
        if see_batch_loss:
            if batch%eval_train == 0:
                print(f'Batch_Loss_{batch} : {loss.item()}')

    if i%eval_train_test==0:
        val_loss, val_acc = test_loop(test_loader)
        print(f'Epoch {i+1}: train_loss: {(total_loss/len(train_loader)):.4f}, train_acc: {(accuracy_score(y_true, y_preds)):.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}')
        


def test_loop(dataset):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        y_true = []
        y_preds = []
        for data, label in dataset:
            data , label = data.to(device), label.to(device)
            logits = model(data)
    
            loss = loss_fn(logits.view(-1), label)
            
            total_loss+=loss.item()
            preds = torch.sigmoid(logits).view(-1) > 0.5
            y_true.extend(label.cpu().tolist())
            y_preds.extend(preds.detach().cpu().tolist())
                              
    return total_loss/len(test_loader), accuracy_score(y_true, y_preds)
    # print(f'val_loss: {total_loss/len(test_loader)}, val_acc: {accuracy_score(y_true, y_preds)}')  

In [315]:
start = time.time()

for i in range(epoch): 
    train_loop(i)
    # break
    
end = time.time()


print(f'Total_time: {end-start}')

Epoch 1: train_loss: 0.3913, train_acc: 0.8105, val_loss: 0.4382, val_acc: 0.8061
Total_time: 24.17899179458618


In [ ]:
# I trained the model on the 5 layer mlp and the result was not that great i got stuck at 80% and I even trained it for 1000 epochs!
# Now I will try to use skip connections, lets see..


#### Lets try a transformer model here!!!!
1. First layer will be layer norm
2. Second will be attention layer
   1. First I will try for a single head attention
   2. Multihead attention
   3. multiblocks


In [355]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [383]:
# Single Head
from torch.nn import functional as F
class sa_layer(nn.Module):
    def __init__(self, channels):
        
        super(sa_layer, self).__init__()
        self.q = nn.Linear(channels, channels // 4, bias = False)
        self.k = nn.Linear(channels, channels // 4, bias = False)
        self.v = nn.Linear(channels, channels // 4, bias = False)

    def forward(self, x):

        q = self.q(x)
        k = self.k(x)

        wei = q @ k.transpose(0,1) * 100  ** -0.5
        wei = F.softmax(wei, dim = -1)

        v = self.v(x)

        out = wei @ v

        return out

        

In [384]:
# Model
torch.manual_seed(42)
class space_finder(nn.Module):
    def __init__(self):
        super(space_finder, self).__init__()
        self.l1 = nn.Linear(12, 128)
        self.ln1 = nn.LayerNorm(128)
        
        self.sa1 = sa_layer(128)
        self.sa2 = sa_layer(128)
        self.sa3 = sa_layer(128)
        self.sa4 = sa_layer(128)
        
        self.ln2 = nn.LayerNorm(128)

        self.ffw = nn.Sequential(
                    nn.Linear(128, 512),
                    nn.ReLU(),
                    nn.Linear(512, 128),
        )

        self.logits = nn.Linear(128, 1)

    def forward(self, x):
        
        x0 = self.l1(x)
        x = self.ln1(x0)
        x1 = self.sa1(x)
        x2 = self.sa2(x)
        x3 = self.sa3(x)
        x4 = self.sa4(x)

        x5 = x0 + torch.cat([x1,x2,x3,x4], dim = -1)
        
        x = self.ln2(x5)

        x = x5 + self.ffw(x)

        logits = self.logits(x)
        
        return logits
        

In [385]:
# Hyperparameter----

batch_size = 64
lr = 1e-3
epoch = 100
batch_eval_inter = 100
eval_train_test = 10
# eval_test = 10

# ------------------

In [386]:
model = space_finder()
model.to(device)

space_finder(
  (l1): Linear(in_features=12, out_features=128, bias=True)
  (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (sa1): sa_layer(
    (q): Linear(in_features=128, out_features=32, bias=False)
    (k): Linear(in_features=128, out_features=32, bias=False)
    (v): Linear(in_features=128, out_features=32, bias=False)
  )
  (sa2): sa_layer(
    (q): Linear(in_features=128, out_features=32, bias=False)
    (k): Linear(in_features=128, out_features=32, bias=False)
    (v): Linear(in_features=128, out_features=32, bias=False)
  )
  (sa3): sa_layer(
    (q): Linear(in_features=128, out_features=32, bias=False)
    (k): Linear(in_features=128, out_features=32, bias=False)
    (v): Linear(in_features=128, out_features=32, bias=False)
  )
  (sa4): sa_layer(
    (q): Linear(in_features=128, out_features=32, bias=False)
    (k): Linear(in_features=128, out_features=32, bias=False)
    (v): Linear(in_features=128, out_features=32, bias=False)
  )
  (ln2): LayerNorm((128,), 

In [387]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [388]:
start = time.time()

for i in range(epoch): 
    train_loop(i)
    # break
    
end = time.time()


print(f'Total_time: {end-start}')

Epoch 1: train_loss: 0.5692, train_acc: 0.7043, val_loss: 0.5146, val_acc: 0.7485
Epoch 11: train_loss: 0.4290, train_acc: 0.7947, val_loss: 0.4322, val_acc: 0.7879
Epoch 21: train_loss: 0.4219, train_acc: 0.7985, val_loss: 0.4404, val_acc: 0.7985
Epoch 31: train_loss: 0.4165, train_acc: 0.7985, val_loss: 0.4297, val_acc: 0.8197
Epoch 41: train_loss: 0.4174, train_acc: 0.7940, val_loss: 0.4229, val_acc: 0.8167
Epoch 51: train_loss: 0.4160, train_acc: 0.7955, val_loss: 0.4222, val_acc: 0.8015
Epoch 61: train_loss: 0.4122, train_acc: 0.7995, val_loss: 0.4198, val_acc: 0.8061
Epoch 71: train_loss: 0.4085, train_acc: 0.8005, val_loss: 0.4157, val_acc: 0.8136
Epoch 81: train_loss: 0.4097, train_acc: 0.8012, val_loss: 0.4226, val_acc: 0.8045
Epoch 91: train_loss: 0.4091, train_acc: 0.8005, val_loss: 0.4087, val_acc: 0.8091
Total_time: 162.90756607055664
